In [18]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [19]:

pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [21]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import time
import datetime as dt

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

In [22]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [23]:
import json
with open('mpd_slice_0_999.json') as json_file:
    data = json.load(json_file)

In [24]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_conc(artists):
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_conc = unique_artists / total_appearances
    return artist_conc

In [37]:
MS_IN_DAY = 86400000
def get_artists_and_dates(uris):
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = []
    dates = []
    tracks = sp.tracks(uris)['tracks']
    for track in tracks:
        if track is None:
            print(track)
        else:
            for artist in track['artists']:
                artists.append(artist['name'])
            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
            dates.append(ms_date / MS_IN_DAY)
            
    return artists, dates

In [31]:
# get all audio features for a given playlist
def get_playlist_audio_features_batch(tracks):
    tracks = pd.DataFrame(tracks)
    playlist_uris = tracks['track_uri']

    remainder = len(tracks) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(tracks):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    remainder = len(tracks) % 50
    artists, dates = [], []
    if remainder > 0:
        artists, dates = get_artists_and_dates(playlist_uris[:remainder])
    while remainder < len(tracks):
        next_artists, next_dates = get_artists_and_dates(playlist_uris[remainder:remainder+50])
        artists += next_artists
        dates += next_dates
        remainder += 50
    return all_features, artists, dates

In [27]:
# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features_v2(tracks):
    all_features, artists, dates = get_playlist_audio_features_batch(tracks)
    all_features = pd.DataFrame(all_features)
    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)
    aggregate_features['date'] = np.mean(dates)
    aggregate_features['date_var'] = np.var(dates)
    aggregate_features['artist_conc'] = get_artist_conc(artists)
    return aggregate_features

In [28]:
# build the aggregate dataset!
def build_dataset_v2(playlists):
    aggregate_dataset = []
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features_v2(playlist['tracks']))
    return aggregate_dataset

In [38]:
dataset = pd.DataFrame(build_dataset_v2(data['playlists'][:10]))
dataset.to_csv(r'csvdata_10.csv')

In [39]:
print(dataset[:2])

   danceability  danceability_var    energy  energy_var  loudness  \
0      0.664077          0.021213  0.781077    0.016335 -4.891212   
1      0.492382          0.018813  0.695923    0.043638 -8.107974   

   loudness_var  speechiness  speechiness_var  acousticness  acousticness_var  \
0      1.721769     0.103698         0.007358      0.083674           0.01420   
1      6.896997     0.091010         0.013898      0.162227           0.04429   

   ...  instrumentalness_var  liveness  liveness_var   valence  valence_var  \
0  ...              0.000013  0.187087      0.021052  0.642750     0.036840   
1  ...              0.130101  0.179344      0.017157  0.476667     0.041661   

        tempo   tempo_var       date   date_var  artist_conc  
0  121.157500  999.439627  13.111522   1.232804     0.746269  
1  124.987128  859.968191   7.900962  41.862441     0.450000  

[2 rows x 21 columns]
